### Imports

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import time
from transformers import Trainer,TrainingArguments,BartConfig,BartTokenizer,BartModel,BartForConditionalGeneration
from transformers import BartTokenizerFast
from torch.utils import data as data_utils
from torch.utils.data import Dataset, DataLoader

import torch

C:\Users\Shoshi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Shoshi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Shoshi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Shoshi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

### Load Data

In [3]:
data_dir = "../DataSets/"

"big_data" contains precomputed simulations for 198 length sequences.

In [138]:
big_data = data_dir+"dna_big_sim_output.txt"
big_data_header = ["dna","length","energy","struct","blank","prob"]

"small_data" is only of length 32, with fewer sequences and is more managable

Future work will be on a more flexible model that can handle variable lengths, but current intended use is only on sequences of a known set length.

In [14]:
small_data = data_dir+"dna_small_sim_output.txt"
small_data_header = ["dna","length","energy","struct"]

Load prefered data set into a pandas dataframe

In [15]:
header_names = small_data_header
data = pd.read_csv(small_data,sep="\\t",header=None,names=header_names)


C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Break the DNA and structure up for the tokenizer 

This is because the tokenizer expects to see words, not individual characters 

(it was made originally for NLP)

(building custom tokenizer may be possible, but model was prebuilt to be used with this tokenizer)

In [16]:
dna = data["dna"]
struct = data["struct"]

dna_list = [" ".join(list(d)) for d in data["dna"]]
struct_list = [" ".join(list(s)) for s in data["struct"]]

length=len(dna[0])

Using same tokenizer model was trained for

In [9]:
tokenizer = BartTokenizerFast.from_pretrained("roberta-base")

### Model and Testing

Load model from trainer output

In [8]:
model_dir = "../Models/"

small_model = model_dir + "DNA_BART_32"
big_model = model_dir + "DNA_BART_198"

In [10]:
model = BartForConditionalGeneration.from_pretrained(small_model, return_dict=True)

This will translate individual sequences or batches of sequences

In [17]:
def translate(dna,model=model,tokenizer=tokenizer,max_length=length+2):#+2 is because models are weird
    # tokenize input
    inputTensor = tokenizer(dna,return_tensors='pt')
    # generate output tensor
    outputTensor = model.generate(inputTensor['input_ids'],
                                 max_length = max_length,
                                  early_stopping=True)
    # decode and clean up output
    output = ["".join(tokenizer.decode(t[2:-1].tolist()).split()) for t in outputTensor]
    return output

For this test, compare the output from translating the first 1000 lines to a pre_run version

In [18]:
output = [translate(d) for d in dna_list[:1000]]

In [19]:
val_file_path = data_dir + "small_bart_output_1000.txt"

with open(val_file_path) as f:
    val_output = f.readlines()
val_output = [l.strip() for l in val_output]

val_output

In [20]:
Z = zip(output,val_output)

comparison = [o == v for o,v in Z]

If everything went right, this should return an empty list

If not, inspect the elements that differ.

In [21]:
np.where(comparison == False)

(array([], dtype=int64),)